In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! conda install -q -y jupyter
! conda install -q -y google-colab -c conda-forge
! python -m ipykernel install --name "py38" --user

In [1]:
# Reload the web page and execute this cell
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.8.0 | packaged by conda-forge | (default, Nov 22 2019, 19:11:38) 
[GCC 7.3.0]


In [14]:
! python -m pip install --upgrade pip
! python -m pip install -q --upgrade ipykernel==5.3.4
! python -m pip install -q --upgrade ipython==7.9.0

In [23]:
# ! python -m pip install pdfplumber
# ! python -m pip install pdf2image
# ! python -m pip install pyPDF2==2.12.1
# ! python -m pip install tqdm
! python -m pip install pdfplumber==0.6.0
! python -m pip install pdf2image
! python -m pip install pyPDF2==2.0.0
! python -m pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.0 MB/s eta 0:00:0000:0100:01
  Created wheel for pdfplumber: filename=pdfplumber-0.6.0-py3-none-any.whl size=33664 sha256=a339a10f4b57d2bc0d5d16ec67295a8d8bac982221378f85c9962da0793dadb9
  Stored in directory: /home/ubuntu/.cache/pip/wheels/42/74/b8/c95f716311b9ee587844cd73381fc19079bccfc05110378c61
Successfully built pdfplumber
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20220524
    Uninstalling pdfminer.six-20220524:
      Successfully uninstalled pdfminer.six-20220524
  Attempting uninstall: pdfplumber
    Found existing installation: pdfplumber 0.5.0
    Uninstalling pdfplumber-0.5.0:
      Successfully uninstalled pdfplumber-0.5.0


In [ ]:
! sudo apt-get install poppler-utils

In [2]:
# Refresh browser in case the change doesn't work
#%cd {folder of the catalog} # For the catalog of the interest
#%cd /content/gdrive/MyDrive/Colab Notebooks/CSU_PhD_Research/CatalogsImagePaper/Misumi    # For Misumi catalog processing
#%cd /content/gdrive/MyDrive/Colab Notebooks/CSU_PhD_Research/CatalogsImagePaper/Newark    # For Newark catalog processing
#%cd /content/gdrive/MyDrive/Colab Notebooks/CSU_PhD_Research/CatalogsImagePaper/Thorlabs  # For Thorlabs catalog processing
#%cd /content/gdrive/MyDrive/Colab Notebooks/CSU_PhD_Research/CatalogsImagePaper/Grainger
# %cd /content/gdrive/MyDrive/Colab Notebooks/CSU_PhD_Research/CatalogsImagePaper/8020
%cd /mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/mcmaster_125_all_pdfs/

/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/mcmaster_125_all_pdfs


In [1]:
import multiprocessing
import argparse
import pdfplumber
import os
from tqdm import tqdm
from pdfminer.layout import LTChar, LTLine
import re
from collections import Counter
import pdf2image
import numpy as np
from PIL import Image, ImageDraw 

from PyPDF2 import PdfFileReader, PdfFileWriter
import time

In [3]:
def create_pdf_pages(pdf_file,numPages,output_dir):
  pdfName = pdf_file.replace('.pdf','_')

  #output_dir = './' + pdfName + 'pdfs'              # Point the output directory
  # if not os.path.exists(output_dir):
  #   os.makedirs(output_dir)

  pdfSource = PdfFileReader(open(pdf_file,"rb"))
  print('PDF source: ', pdfSource)
  if numPages == 'all':
    numPages = pdfSource.numPages
  
  for page in tqdm(range(numPages)):
    print('\nGenerating the individual pages from main catalogue.\n')
    # ipdb.set_trace(context=10)
    time.sleep(0.1)
    output = PdfFileWriter()
    output.addPage(pdfSource.getPage(page))

    with open(output_dir+'/'+pdfName+"%s.pdf" % page, "wb") as outputStream:
      output.write(outputStream)
    del output
    outputStream.close

In [5]:
def within_bbox(bbox_bound, bbox_in):
  assert bbox_bound[0] <= bbox_bound[2]
  assert bbox_bound[1] <= bbox_bound[3]
  assert bbox_in[0] <= bbox_in[2]
  assert bbox_in[1] <= bbox_in[3]

  x_left = max(bbox_bound[0], bbox_in[0])
  y_top = max(bbox_bound[1], bbox_in[1])
  x_right = min(bbox_bound[2], bbox_in[2])
  y_bottom = min(bbox_bound[3], bbox_in[3])
  if x_right < x_left or y_bottom < y_top:
    return False
  
  intersection_area = (x_right - x_left) * (y_bottom - y_top)
  bbox_in_area = (bbox_in[2] - bbox_in[0]) * (bbox_in[3] - bbox_in[1])
  if bbox_in_area == 0:
    return False
  
  iou = intersection_area / float(bbox_in_area)
  return iou > 0.98

In [6]:
def clear_word_box(word,objs,regex_clean):
  # detects ... r'(?:\.){2,}' check https://regex101.com/
  if re.search(regex_clean,word): 
    # Clear the regex item from the word and equate to cleared_word
    cleared_word = re.sub(regex_clean,'',word)
    # Find the new bounding box of this word
    word_bbox = []
  else:
    print('Nothing to clear!')
  return cleared_word, word_bbox

In [7]:
def find_index_of_all_char(s, char):
    return [i for i, ltr in enumerate(s) if ltr == char]
  
# Alternative implementation
# s = 'A...ABC...'
# c = '.'
# idx = []
# # Find the index of the character
# for i in range(len(s)):
#   s_temp = s
#   if c==s_temp[i]:
#     idx.append(i)

In [8]:
# remove character bounding box from the word bounding box
def remove_char_bbox(word_bbox, char_bbox, char_counter, idx_char):
    # ipdb.set_trace(context=10)
    # print('word_bbox',word_bbox)
    # print('char_bbox',char_bbox)
    if idx_char == 0 and char_counter == 0: # if it is the first char
        word_bbox[0] = char_bbox[2]
    elif char_counter > 0 and char_counter < len(word_bbox): # the bound box is in the middle of the bounding box
        word_bbox[0] = char_bbox[2] 
    elif idx_char > char_counter: # last char in the word
        word_bbox[2] = char_bbox[0]
    else:
        print('Something went wrong!')
    return word_bbox

In [9]:
def annotate_all(word_text, word_bbox, anno_color, anno_img, tokens, 
                 pdf_width, pdf_height, target_width, target_height,
                 pdf_file, output_dir):
    # Format word_bbox
    f_x0 = int(word_bbox[0] / pdf_width * target_width)
    f_y0 = int(word_bbox[1] / pdf_height * target_height)
    f_x1 = int(word_bbox[2] / pdf_width * target_width)
    f_y1 = int(word_bbox[3] / pdf_height * target_height)
    # Normalized units target_width/width (image/pdf width)
    word_bbbox = tuple([f_x0, f_y0, f_x1, f_y1])

    # Plot Annotation
    x0, y0, x1, y1 = word_bbbox
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    if(x0>=pdf_width):
        x0 = pdf_width-1
    if(x1>=pdf_width):
        x1 = pdf_width-1
    if(y0>=pdf_height):
        y0=pdf_height-1
    if(y1>=pdf_height):
        y1=pdf_height-1

    # anno_color = [255, 0, 0]                        # Red color for text
    for x in range(x0, x1-1):
        anno_img[x, y0] = anno_color
    for x in range(x0, x1-1):
        anno_img[x, y1] = anno_color
    for y in range(y0, y1-1):
        anno_img[x0,y] = anno_color
    for y in range(y0, y1-1):
        anno_img[x1,y] = anno_color

    with Image.open(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_annoverlay.png')) as im:
        draw = ImageDraw.Draw(im)
        draw.rectangle(word_bbbox,outline=tuple(anno_color))
        # write to stdout
        im.save(os.path.join(output_dir, pdf_file.replace('.pdf','')+'_annoverlay.png'))
    
    word_bbbox = tuple([str(t) for t in word_bbbox])
    fontname = 'default'
    tokens.append((word_text,) + word_bbbox + (fontname,))
    return tokens, anno_img

In [8]:
idx_counter = 6
test = '0.45'
regex_clean = r"(?:\.){2,}" # Clear unwanted pattern from the word set (Check Reg 101 the pattern)
# Find the idx and string of deliniation patterns (move to a helper function)
obj_skip_idx_str = [(g.start(), g.group()) for g in re.compile(regex_clean).finditer(test)]
obj_skip_idx_start = [i[0] for i in obj_skip_idx_str]
obj_skip_idx_end = [i[0]+len(i[1]) for i in obj_skip_idx_str]
obj_skip_str = [i[1] for i in obj_skip_idx_str]
# Create a list of range of the deliniation patterns and unpack them (move to a helper function)
obj_range_list = [[*range(x,y,1)] for (x, y) in zip(obj_skip_idx_start,obj_skip_idx_end)]  
obj_combined_list = [y for x in obj_range_list for y in x]

if idx_counter in obj_combined_list:
    print('Skipping object')

In [10]:
def create_data(idx, pdf_file, data_dir, output_dir):
    
    target_width = 762
    target_height = 1000
    unwanted_character = '.'    # remove unwanted character from the word (. used for delinieation in the catalog)
    regex_clean = r"^\.+|\.(?!\d)|\.+$"

    image_path = os.path.join(data_dir.replace('_pdfs','_images'), f'{pdf_file.replace(".pdf","_ori.jpg")}')
    if(os.path.exists(image_path)):
      print('\nImage:', image_path,' exists.\n')
      return
    
    else:
      print('\nImage:'+data_dir.replace('_pdfs','_images')+'/'+pdf_file.replace('.pdf','_ori.jpg')+' needs to be annotated.\n')
      
      # Convert pdf to image
      try:
          pdf_images = pdf2image.convert_from_path(os.path.join(data_dir, pdf_file),size=(target_width,target_height))
      except:
          print("Failed to convert.")
          return

      # Import pdf pages
      page_tokens = []
      try:
          pdf = pdfplumber.open(os.path.join(data_dir, pdf_file))
      except:
          print("Failed pdf generation.")
          return

      # Show the loop with a progress bar via tqdim
      for page_id in range(len(pdf.pages)): # This runs for single or multiple pages
          # print('\n...Generating the annotations.')

          # Create original image
          pdf_images[page_id].save(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_ori.jpg'))
          pdf_images[page_id].save(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_annoverlay.png'))
          
          # if os.path.exists(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_ori.jpg')):
          #   continue
          # Prepare Words/ Tokens
          tokens = []
          this_page = pdf.pages[page_id]
          pdf_width = int(this_page.width)
          pdf_height = int(this_page.height)
          words = this_page.extract_words(x_tolerance=1.75 , y_tolerance=1.65, use_text_flow=True)
          # words = this_page.extract_words(x_tolerance=1.77 , y_tolerance=1.77, use_text_flow=False)
          # anno_img = np.ones([int(this_page.width), int(this_page.height)] + [3], dtype=np.uint8) * 255 # in pdf dimensions
          anno_img = np.ones([int(pdf_width), int(pdf_height)] + [3], dtype=np.uint8) * 255 # in pdf dimensions
          
          # Processing words
          for word in words:
            skip_word = False
            # print('word',word['text'])
            # if os.path.exists(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_ori.jpg')):
            #   continue
            # Check the word for any unwanted patterns and skip if the word is build by it for example '....' as a word
            if re.search(regex_clean,word['text']):
              # print('word to skip',word['text'])
              if (word['text'].count(unwanted_character) == len(word['text'])):
                continue

            word_bbox_org = [float(word['x0']), float(word['top']), float(word['x1']), float(word['bottom'])]    # Original bounding box to use in comparison
            word_bbox = [float(word['x0']), float(word['top']), float(word['x1']), float(word['bottom'])]        # Bounding box for words to change and later store
            word_bbbox = tuple([int(word['x0']), int(word['top']), int(word['x1']), int(word['bottom'])])
            
            # consequitivedots = re.compile(r'\.{3,}')
            # word_text = consequitivedots.sub('.', word['text']).strip()
            # Clear a string of unwanted pattern regex_clean.      
            word_text = re.sub(r"\.{2,}", '.', word['text']).strip()
            word_text = re.sub(regex_clean, '', word_text)                                                       # Clear a string of unwanted pattern regex_clean.

            if re.search(regex_clean,word['text']): 
              skip_word = True
              idx_counter = 0
              idx_reverse_counter = 0
              # Extract characters from the layout objects and create new set of objects
              objs = []
              #idx_char = find_index_of_all_char(word['text'],'.')       # Find the index of the character
              splitted_words = re.split(regex_clean,word['text'])       # Clear a string of unwanted characters
              
              # if splitted words has an empty string pop that
              if ('' in splitted_words) :
                splitted_words.pop(splitted_words.index(''))              # Pop the empty item from splitted_words

              if (len(splitted_words) > 1):
                splitted_words_str = (" ".join([str(item) for item in splitted_words]))
                splitted_words_start_end = [(m.start(0), m.end(0)) for m in re.finditer(' ', splitted_words_str)]
              
              # Find the idx and string of deliniation patterns (move to a helper function)
              obj_skip_idx_str = [(g.start(), g.group()) for g in re.compile(regex_clean).finditer(word['text'])]
              obj_skip_idx_start = [i[0] for i in obj_skip_idx_str]
              obj_skip_idx_end = [i[0]+len(i[1]) for i in obj_skip_idx_str]
              obj_skip_str = [i[1] for i in obj_skip_idx_str]
              # Create a list of range of the deliniation patterns and unpack them (move to a helper function)
              obj_range_list = [[*range(x,y,1)] for (x, y) in zip(obj_skip_idx_start,obj_skip_idx_end)]  
              obj_combined_list = [y for x in obj_range_list for y in x]

              # print('-') # Marker for the debugging point for the new word
              
              for splitted_word in splitted_words:
                if splitted_word == '':
                  continue
                first_char_marked = True
                char_counter = 0
                
                
                if (len(splitted_words) > 1):
                  # Splitted word check
                  if splitted_words.index(splitted_word)==0:
                    start_idx_splitted_word_char = 0
                    end_idx_splitted_word_char = start_idx_splitted_word_char + len(splitted_word) - 1
                  else:
                    start_idx_splitted_word_char = splitted_words_start_end[splitted_words.index(splitted_word)-1][0] # Starting index of splitted words
                    end_idx_splitted_word_char = start_idx_splitted_word_char + len(splitted_word) - 1
                else:
                  start_idx_splitted_word_char = 0
                  end_idx_splitted_word_char = start_idx_splitted_word_char + len(splitted_word) - 1
                  
                count_down = start_idx_splitted_word_char

                for obj in this_page.layout._objs: # In efficient but works
                  
                  if not isinstance(obj, LTChar) or obj.get_text() == ' ':
                    continue
                  
                  obj_bbox = (obj.bbox[0], float(this_page.height) - obj.bbox[3],
                              obj.bbox[2], float(this_page.height) - obj.bbox[1])

                  if within_bbox(word_bbox_org, obj_bbox):
                    
                    # To map the obj start point to word start point
                    if (len(objs) > 0 and idx_reverse_counter != 0):
                      idx_reverse_counter -= 1
                      continue
                    if idx_counter in obj_combined_list:
                      idx_counter += 1
                      continue

                    if char_counter == 0:                                 # Start of the word
                      word_bbox[0] = obj_bbox[0]                          # Adjust bbox x value
                    elif start_idx_splitted_word_char > 0 and char_counter == 0:
                      word_bbox[0] = obj_bbox[0]
                    elif char_counter == (len(splitted_word) - 1):        # End of the word
                      word_bbox[2] = obj_bbox[2]                          # adjust bbox x value
                    
                    if len(splitted_word) == 1:                           # Single character string
                      word_bbox[0] = obj_bbox[0]                          # Adjust bbox x value
                      word_bbox[2] = obj_bbox[2]

                    idx_counter += 1
                    char_counter += 1
                    objs.append(obj)

                    if char_counter == len(splitted_word):
                      # print('Cleared word:', splitted_word)
                      # print('Length of the splitted word:', len(splitted_word))
                      color = [255, 0, 0] # (RED)
                      tokens, anno_img = annotate_all(splitted_word, word_bbox, color, anno_img, tokens, 
                                                      pdf_width, pdf_height, target_width, target_height,
                                                      pdf_file, output_dir)
                      idx_reverse_counter = idx_counter
                      break
            
              # Extract the name of the font from the object
              fontname = []
              for obj in objs:
                fontname.append(obj.fontname)
              if len(fontname) != 0:
                c = Counter(fontname)
                fontname, _ = c.most_common(1)[0]
              else:
                fontname = 'default'
                
            if(skip_word == False):
              color = [255, 0, 0] # (RED)
              tokens, anno_img = annotate_all(word_text, word_bbox, color, anno_img, tokens, 
                                            pdf_width, pdf_height, target_width, target_height,
                                            pdf_file, output_dir)

          for image in this_page.images:
              image_bbox = (float(image['x0']), 
                            float(image['top']), 
                            float(image['x1']), 
                            float(image['bottom']))

              word_text = '##LTFigure##'
              anno_color = [0, 255, 0] # (GREEN)
              tokens, anno_img = annotate_all(word_text, image_bbox, anno_color, anno_img, tokens, 
                                              pdf_width, pdf_height, target_width, target_height,
                                              pdf_file, output_dir)
      
          anno_img = np.swapaxes(anno_img, 0, 1)
          anno_img = Image.fromarray(anno_img, mode='RGB')
          page_tokens.append((page_id, tokens, anno_img))
          anno_img.save(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_ann.jpg'))

          with open(os.path.join(output_dir, pdf_file.replace('.pdf', '')+'_{}.txt'.format(str(page_id))),'w',encoding='utf8') as fp:
              for token in tokens:
                  fp.write('\t'.join(token) + '\n')

In [14]:
%cd /mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/Paper_Supporting_Test/
!pwd

/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/Paper_Supporting_Test
/mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/Paper_Supporting_Test


In [11]:
!rm -rf /mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/McMasterCarrData100_pdfs
!rm -rf /mnt/data/Data-Document/Data_CatalogBankPaper/McMasterCarr/McMasterCarrData100_pdfs_images

In [15]:
os.listdir()

['mcmaster-125_3378_1014.pdf',
 'mcmaster-125_3378_1029.pdf',
 'mcmaster-125_3378_1000.pdf',
 'mcmaster-125_3378_1001.pdf',
 'mcmaster-125_3378_1002.pdf',
 'mcmaster-125_3378_1003.pdf',
 'mcmaster-125_3378_1004.pdf',
 'mcmaster-125_3378_1005.pdf',
 'mcmaster-125_3378_1006.pdf',
 'mcmaster-125_3378_1007.pdf',
 'mcmaster-125_3378_1008.pdf',
 'mcmaster-125_3378_1009.pdf',
 'mcmaster-125_3378_1010.pdf',
 'mcmaster-125_3378_1011.pdf',
 'mcmaster-125_3378_1012.pdf',
 'mcmaster-125_3378_1013.pdf',
 'mcmaster-125_3378_1015.pdf',
 'mcmaster-125_3378_1016.pdf',
 'mcmaster-125_3378_1017.pdf',
 'mcmaster-125_3378_1018.pdf',
 'mcmaster-125_3378_1019.pdf',
 'mcmaster-125_3378_1020.pdf',
 'mcmaster-125_3378_1021.pdf',
 'mcmaster-125_3378_1022.pdf',
 'mcmaster-125_3378_1023.pdf',
 'mcmaster-125_3378_1024.pdf',
 'mcmaster-125_3378_1025.pdf',
 'mcmaster-125_3378_1026.pdf',
 'mcmaster-125_3378_1027.pdf',
 'mcmaster-125_3378_1028.pdf',
 'mcmaster-125_3378_1030.pdf',
 'mcmaster-125_3378_1031.pdf',
 'mcmast

In [16]:
if __name__ == '__main__':
  # Generate data and catalog should be in the same directory
  catalogName = 'mcmaster-125_3378.pdf'
  # catalogName = '8020_Catalog_23_70.pdf'
  # catalogName = 'DigiKeyUKCatalogue.pdf'
  # catalogName = 'V21_1_Optomechanics.pdf'
  catalogName = catalogName.replace('.pdf','')

  # Location of the folders
  datadir = './' + catalogName+'_all_pdfs'               # Enter where the pdf files are
  if not os.path.exists(datadir):
    os.makedirs(datadir)
    # STEP#1 Convert the catalog to an individual paged pdf document and save it
    create_pdf_pages(catalogName+'.pdf','all',datadir)

  # Extract the list of files and create a string array with .pdf suffix one
  os.listdir(datadir) # For some reason the next line results in an error without running this line first
  pdf_folder = list(os.listdir(datadir))
  
  pdf_files = [t for t in pdf_folder if t.endswith('.pdf')] 
  print('Number of pdf files:',len(pdf_files))
  output_dir = './' + catalogName + '_all_pdfs_images'         # Point the output directory
  # output_dir = './'
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)
  
  # Reference for parallel processing:
  # https://stackoverflow.com/questions/8533318/multiprocessing-pool-when-to-use-apply-apply-async-or-map
  pool = multiprocessing.Pool(processes=10) ## Enable multiprocessor
  # print("Number of cpu : ", multiprocessing.cpu_count())
  print('\n')
  # Show for loop progress with tqdm
  for idx, pdf_file in enumerate(tqdm(pdf_files)):
      basefilename = pdf_file.replace('.pdf', '')
      if not os.path.exists(output_dir):
        os.makedirs(output_dir) 
      ori_jpg_file = os.path.join(output_dir, f'{basefilename}_ori.jpg')
      if os.path.exists(ori_jpg_file):
          # print(f"\n{ori_jpg_file} file exists for {pdf_file}\n")
          continue
        
      print("\nData directory:" + datadir)    
      print("Pdf File:" + pdf_file)          
      # create_data(idx, pdf_file, datadir, output_dir)
      pool.apply_async(create_data, (idx, pdf_file, datadir, output_dir))    # multiprocessing
  
  pool.close()
  pool.join()

FileNotFoundError: [Errno 2] No such file or directory: 'mcmaster-125_3378.pdf'